In [1]:
!pip -q install -U scikit-image
!pip -q install einops
!pip -q install lmdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip -q install thefuzz
from thefuzz import fuzz
from thefuzz import process

In [2]:
#Load model and weights
!gdown 1VU-qucVHcNbu2qnGkWWNwO0TRhY-7iH2
!unzip -q vietocr.zip
!unzip -q weights.zip

#Import model
from vietocr import *

Downloading...
From: https://drive.google.com/uc?id=1VU-qucVHcNbu2qnGkWWNwO0TRhY-7iH2
To: /content/vietocr.zip
100% 142M/142M [00:01<00:00, 101MB/s]


In [ ]:
#Load publictest
!gdown 1b2_B1HsssTCFBtLMG9xzndxmygM3TLCF
!unzip -q public_test.zip

In [40]:
#Load config
def load_config(yml_path):
  with open(yml_path, "r") as stream:
    try:
      config = yaml.safe_load(stream)
      return config
    except yaml.YAMLError as exc:
      print(exc)

#Preprocessing
def remove_background(image):
  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  # Remove horizontal lines
  horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
  remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
  cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  for c in cnts:
    cv2.drawContours(thresh, [c], -1, (0,255,255), 5)
  # Remove vertical lines
  vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,40))
  remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
  cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  for c in cnts:
    cv2.drawContours(thresh, [c], -1, (0,255,255), 15)
  contours,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
  bounding_boxes = []
  for cnt in contours:
    if cv2.contourArea(cnt)>50:
      [x,y,w,h] = cv2.boundingRect(cnt)
      if (x, y) != (0, 0) and (y+h-y)/image.shape[0] > 0.18 and (x+w-x)/image.shape[1] > 0.01 and (x+w-x)/(y+h-y) > 0.15:
        bounding_boxes.append([x,y,x+w,y+h])
  if np.array(bounding_boxes).size != 0:
    yymin = min(np.array(bounding_boxes)[:, 1])
    xxmax = max(np.array(bounding_boxes)[:, 2])
    yymax = max(np.array(bounding_boxes)[:, 3])
    xxmax = xxmax + 40 if xxmax + 40 < image.shape[1] else xxmax
    yymin = yymin - 5 if yymin - 5 > 0 else yymin
    yymax = yymax + 5 if yymax + 5 < image.shape[0] else yymax
    return image[yymin:yymax, 0:xxmax]
  else:
    return image

#Inference
def prediction(img_path):
  img = cv2.imread(img_path)
  processed_img = remove_background(img)
  img = Image.fromarray(processed_img)
  s = detector.predict(img, return_prob = False)
  return s

def predict(image_folder, output_file_path):
    prediction = pd.DataFrame(columns=['id', 'answer', 'elapsed_time'])
    index = 0
    for person_id in os.listdir(image_folder):
        for image_id in os.listdir(os.path.join(image_folder, person_id)):
            fp = os.path.join(image_folder, person_id, image_id)
            image_id = os.path.join(person_id, image_id)
            image = cv2.imread(fp)

            # Start inference
            start = time.time()
            processed_img = remove_background(image) # preprocess
            img = Image.fromarray(processed_img)
            answer = s = detector.predict(img, return_prob = False) # infer
            answer = answer.replace('Đp', 'Đg').replace('đp', 'đg') # post process
            end = time.time()

            prediction.loc[index] = [image_id, answer, end - start]
            index += 1
    # Write prediction
    prediction.to_csv(output_file_path, index=False)

#Inference without using quantization

In [ ]:
#Update config
config = load_config('/content/base.yml')
config['weights'] = '/content/weights/transformerocr.pth' #trained on our modified dataset

#Load predictor
detector = Predictor(config)

In [ ]:
import pandas as pd
predict("/content/public_test/images", "team_00_private_test_pred.csv")
pd.read_csv('/content/team_00_private_test_pred.csv')[['id', 'answer']].to_csv('private_test_pred.csv', index = False)

In [ ]:
def check_modelsize(config):
  model, vocab = build_model(config)
  weights = config['weights']
  model.load_state_dict(torch.load(weights, map_location=torch.device(config['device'])))
  param_size = 0
  for param in model.parameters():
      param_size += param.nelement() * param.element_size()
  buffer_size = 0
  for buffer in model.buffers():
      buffer_size += buffer.nelement() * buffer.element_size()
  size_all_mb = (param_size + buffer_size) / 1024**2
  print('model size: {:.3f}MB'.format(size_all_mb))

check_modelsize(config)

model size: 144.666MB


##Inference using quantization

In [42]:
class QuantizedCNN(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedCNN, self).__init__()
        self.quant = torch.quantization.QuantStub() # QuantStub converts tensors from floating point to quantized. This will only be used for inputs.
        self.dequant = torch.quantization.DeQuantStub() # DeQuantStub converts tensors from quantized to floating point. This will only be used for outputs.
        self.model_fp32 = model_fp32 # FP32 model
    def forward(self, x):
        x = self.quant(x) #manually specify where tensors will be converted from floating point to quantized in the quantized model
        x = self.model_fp32(x)
        x = self.dequant(x) #manually specify where tensors will be converted from quantized to floating point in the quantized model
        return x

def build_quantized_model(config):
  model, vocab = build_model(config)
  model = model.eval()
  for m in model.cnn.model.modules():
      if type(m) == nn.Sequential:
          for n, layer in enumerate(m):
              if type(layer) == nn.Conv2d:
                  torch.quantization.fuse_modules(m, [str(n), str(n + 1), str(n + 2)], inplace=True)
  quantized_cnn = QuantizedCNN(model_fp32=model.cnn)
  quantized_cnn.qconfig = torch.quantization.get_default_qconfig("fbgemm")
  quantized_cnn = torch.quantization.prepare_qat(quantized_cnn, inplace=True)
  quantized_cnn = quantized_cnn.to(torch.device('cpu'))
  model.cnn = torch.quantization.convert(quantized_cnn, inplace=True)
  return model, vocab

class Predictor():
    def __init__(self, config, quantize = False):
        device = config['device']
        if quantize:
            model, vocab = build_quantized_model(config)
            model.load_state_dict(torch.load(config['quantize_weights']), strict=False)
        else:
            model, vocab = build_model(config)
            model.load_state_dict(torch.load(config['weights'], map_location=torch.device(device)))
        self.config = config
        self.model = model
        self.vocab = vocab
        self.device = device
    def predict(self, img, return_prob=False):
        img = process_input(img, self.config['dataset']['image_height'],
                self.config['dataset']['image_min_width'], self.config['dataset']['image_max_width'])
        img = img.to(self.config['device'])
        if self.config['predictor']['beamsearch']:
            sent = translate_beam_search(img, self.model)
            s = sent
            prob = None
        else:
            s, prob = translate(img, self.model)
            s = s[0].tolist()
            prob = prob[0]
        s = self.vocab.decode(s)
        if return_prob:
            return s, prob
        else:
            return s
    def predict_batch(self, imgs, return_prob=False):
        bucket = defaultdict(list)
        bucket_idx = defaultdict(list)
        bucket_pred = {}
        sents, probs = [0]*len(imgs), [0]*len(imgs)
        for i, img in enumerate(imgs):
            img = process_input(img, self.config['dataset']['image_height'],
                self.config['dataset']['image_min_width'], self.config['dataset']['image_max_width'])
            bucket[img.shape[-1]].append(img)
            bucket_idx[img.shape[-1]].append(i)
        for k, batch in bucket.items():
            batch = torch.cat(batch, 0).to(self.device)
            s, prob = translate(batch, self.model)
            prob = prob.tolist()
            s = s.tolist()
            s = self.vocab.batch_decode(s)
            bucket_pred[k] = (s, prob)
        for k in bucket_pred:
            idx = bucket_idx[k]
            sent, prob = bucket_pred[k]
            for i, j in enumerate(idx):
                sents[j] = sent[i]
                probs[j] = prob[i]
        if return_prob:
            return sents, probs
        else:
            return sents

In [31]:
!gdown 1VFpWftC5ECR3--GLWlwfsdcN3s-hxLbp

Downloading...
From: https://drive.google.com/uc?id=1VFpWftC5ECR3--GLWlwfsdcN3s-hxLbp
To: /content/quantize_transformerocr.pth
100% 91.4M/91.4M [00:08<00:00, 11.4MB/s]


In [43]:
#Update config
config = load_config('./base.yml')
config['device'] = 'cpu'
config['cnn']['pretrained']=False
config['quantize_weights'] = "./quantize_transformerocr.pth"

In [51]:
# create detector
detector = Predictor(config, quantize = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use qu

In [52]:
import pandas as pd
predict("/content/public_test/images", "team_00_private_test_pred.csv")
pd.read_csv('/content/team_00_private_test_pred.csv')[['id', 'answer']].to_csv('private_test_pred.csv', index = False)

In [57]:
def check_modelsize(config):
  model, vocab = build_quantized_model(config)
  model.load_state_dict(torch.load(config['quantize_weights']), strict=False)
  param_size = 0
  for param in model.parameters():
      param_size += param.nelement() * param.element_size()
  buffer_size = 0
  for buffer in model.buffers():
      buffer_size += buffer.nelement() * buffer.element_size()
  size_all_mb = (param_size + buffer_size) / 1024**2
  print('model size: {:.3f}MB'.format(size_all_mb))

check_modelsize(config)

model size: 67.694MB
